Importing the required libraries

In [222]:
import pandas as pd
import os
import requests
from io import BytesIO
import numpy as np
from datetime import timedelta
from datetime import datetime

Loading the initial datasets

In [224]:
url_transactions = "https://github.com/Cnj31/Applied-Research/raw/refs/heads/main/Transaction_100K_Dataset.xlsx"
url_ads = "https://github.com/Cnj31/Applied-Research/raw/refs/heads/main/Ad_Influence_100K_Dataset.xlsx"
url_social = "https://github.com/Cnj31/Applied-Research/raw/refs/heads/main/Social_Media_Sessions_100K.xlsx"

# Download and read each file
df_transactions = pd.read_excel(BytesIO(requests.get(url_transactions).content))
df_ads = pd.read_excel(BytesIO(requests.get(url_ads).content))
df_social = pd.read_excel(BytesIO(requests.get(url_social).content))

# Check loaded shapes
print("Transactions:", df_transactions.shape)
print("Ads:", df_ads.shape)
print("Social:", df_social.shape)

Transactions: (100000, 7)
Ads: (100000, 5)
Social: (100000, 4)


FEATURE ENGINEERING - TRANSACTION DATA

Inluding all days in a week randomly

In [227]:
# Making a copy
df_transactions_sim = df_transactions.copy()

# Convert column to datetime
df_transactions_sim["Transaction_DateTime"] = pd.to_datetime(df_transactions_sim["Transaction_DateTime"])

# Create random day shifts between 0 and 29 days
np.random.seed(42)
date_shifts = np.random.randint(0, 30, size=len(df_transactions_sim))

# Apply shift to simulate a date spread over a month
df_transactions_sim["Transaction_DateTime"] = df_transactions_sim["Transaction_DateTime"] + pd.to_timedelta(date_shifts, unit="d")

# Preview results
df_transactions_sim[["User_ID", "Transaction_DateTime"]].head()

,User_ID,Transaction_DateTime
0,1102,2023-05-28 14:29:00
1,1435,2023-06-10 13:42:00
2,1860,2023-06-19 08:33:00
3,1270,2023-06-05 10:05:00
4,1106,2023-06-01 08:44:00


In [228]:
df_transactions_sim.head()

,User_ID,Transaction_DateTime,Amount,Category,Transaction_Type,Impulse_Score,Is_Impulse
0,1102,2023-05-28 14:29:00,50.07,Fashion,Card,0.10,0
1,1435,2023-06-10 13:42:00,37.92,Travel,Wallet,0.66,0
2,1860,2023-06-19 08:33:00,29.60,Entertainment,Wallet,0.36,0
3,1270,2023-06-05 10:05:00,23.06,Entertainment,Wallet,0.85,1
4,1106,2023-06-01 08:44:00,34.91,Entertainment,Card,0.08,0


Adding feature engineering columns

In [230]:
# Assuming df_transactions_sim is your updated transaction DataFrame
df = df_transactions_sim.copy()

# Ensure Transaction_DateTime is in datetime format
df["Transaction_DateTime"] = pd.to_datetime(df["Transaction_DateTime"])

# 1. Day of the week (e.g., Monday, Tuesday)
df["Day_of_Week"] = df["Transaction_DateTime"].dt.day_name()

# 2. Time of Day Bucket
def get_time_of_day(hour):
    if 5 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

df["Hour"] = df["Transaction_DateTime"].dt.hour
df["Time_of_Day"] = df["Hour"].apply(get_time_of_day)

# 3. Is Weekend (Saturday/Sunday = 1)
df["Is_Weekend"] = df["Transaction_DateTime"].dt.weekday >= 5
df["Is_Weekend"] = df["Is_Weekend"].astype(int)

# 4. Days Since Salary (salary paid on 1st of the month)
df["Salary_Date"] = df["Transaction_DateTime"].apply(lambda x: datetime(x.year, x.month, 1))
df["Days_Since_Salary"] = (df["Transaction_DateTime"] - df["Salary_Date"]).dt.days

# Drop temporary column
df.drop(columns=["Hour"], inplace=True)

# Preview
df[["Transaction_DateTime", "Day_of_Week", "Time_of_Day", "Is_Weekend", "Days_Since_Salary"]].head()

,Transaction_DateTime,Day_of_Week,Time_of_Day,Is_Weekend,Days_Since_Salary
0,2023-05-28 14:29:00,Sunday,Afternoon,1,27
1,2023-06-10 13:42:00,Saturday,Afternoon,1,9
2,2023-06-19 08:33:00,Monday,Morning,0,18
3,2023-06-05 10:05:00,Monday,Morning,0,4
4,2023-06-01 08:44:00,Thursday,Morning,0,0


Adding post salary window column

In [232]:
df["Post_Salary_Window"] = df["Days_Since_Salary"].apply(lambda x: 1 if 0 <= x <= 3 else 0)

In [233]:
df.head()

,User_ID,Transaction_DateTime,Amount,Category,Transaction_Type,Impulse_Score,Is_Impulse,Day_of_Week,Time_of_Day,Is_Weekend,Salary_Date,Days_Since_Salary,Post_Salary_Window
0,1102,2023-05-28 14:29:00,50.07,Fashion,Card,0.10,0,Sunday,Afternoon,1,2023-05-01,27,0
1,1435,2023-06-10 13:42:00,37.92,Travel,Wallet,0.66,0,Saturday,Afternoon,1,2023-06-01,9,0
2,1860,2023-06-19 08:33:00,29.60,Entertainment,Wallet,0.36,0,Monday,Morning,0,2023-06-01,18,0
3,1270,2023-06-05 10:05:00,23.06,Entertainment,Wallet,0.85,1,Monday,Morning,0,2023-06-01,4,0
4,1106,2023-06-01 08:44:00,34.91,Entertainment,Card,0.08,0,Thursday,Morning,0,2023-06-01,0,1


Moving impulse columns to the end

In [235]:
impulse_cols = df[["Is_Impulse", "Impulse_Score"]]
df.drop(columns=["Is_Impulse", "Impulse_Score"], inplace=True)
df = pd.concat([df, impulse_cols], axis=1)
df.head()

,User_ID,Transaction_DateTime,Amount,Category,Transaction_Type,Day_of_Week,Time_of_Day,Is_Weekend,Salary_Date,Days_Since_Salary,Post_Salary_Window,Is_Impulse,Impulse_Score
0,1102,2023-05-28 14:29:00,50.07,Fashion,Card,Sunday,Afternoon,1,2023-05-01,27,0,0,0.10
1,1435,2023-06-10 13:42:00,37.92,Travel,Wallet,Saturday,Afternoon,1,2023-06-01,9,0,0,0.66
2,1860,2023-06-19 08:33:00,29.60,Entertainment,Wallet,Monday,Morning,0,2023-06-01,18,0,0,0.36
3,1270,2023-06-05 10:05:00,23.06,Entertainment,Wallet,Monday,Morning,0,2023-06-01,4,0,1,0.85
4,1106,2023-06-01 08:44:00,34.91,Entertainment,Card,Thursday,Morning,0,2023-06-01,0,1,0,0.08


In [236]:
df.to_excel("Transformed_Transaction_Data.xlsx", index=False)

FEATURE ENGINEERING - AD INFLUENCE DATA

In [238]:
# Ensure datetime columns are in correct format
df_ads["Ad_Timestamp"] = pd.to_datetime(df_ads["Ad_Timestamp"])
df_ads["Transaction_Timestamp"] = pd.to_datetime(df_ads["Transaction_Timestamp"])

# 1. Time difference in minutes between ad view and transaction
df_ads["Time_Diff_Minutes"] = (df_ads["Transaction_Timestamp"] - df_ads["Ad_Timestamp"]).dt.total_seconds() / 60

# 2. Within Influence Window (1 if transaction happened within 90 mins after ad)
df_ads["Within_Influence_Window"] = df_ads["Time_Diff_Minutes"].apply(lambda x: 1 if 0 <= x <= 90 else 0)

# 3. Category Match (1 if Transaction_Category in Ad_Content)
df_ads["Category_Match"] = df_ads.apply(
    lambda row: 1 if row["Transaction_Category"].lower() in row["Ad_Content"].lower() else 0,
    axis=1
)

# Preview the updated DataFrame
df_ads.head()

,User_ID,Ad_Timestamp,Transaction_Timestamp,Ad_Content,Transaction_Category,Time_Diff_Minutes,Within_Influence_Window,Category_Match
0,1102,2023-05-22 12:42:00,2023-05-22 14:29:00,Food Promo,Fashion,107.0,0,0
1,1435,2023-05-22 11:47:00,2023-05-22 13:42:00,Food Promo,Travel,115.0,0,0
2,1860,2023-05-22 08:01:00,2023-05-22 08:33:00,Travel Promo,Entertainment,32.0,1,0
3,1270,2023-05-22 09:28:00,2023-05-22 10:05:00,Health Promo,Entertainment,37.0,1,0
4,1106,2023-05-22 08:03:00,2023-05-22 08:44:00,Entertainment Promo,Entertainment,41.0,1,1


In [239]:
df_ads.to_excel("Transformed_Ad_Influence_Dataset.xlsx", index=False)

FEATURE ENGINEERING - SOCIAL MEDIA SESSION DATA

In [241]:
# Ensure datetime columns are in correct format
df_social["Session_Start"] = pd.to_datetime(df_social["Session_Start"])
df_social["Session_End"] = pd.to_datetime(df_social["Session_End"])

# 1. Session Duration in minutes
df_social["Session_Duration_Min"] = (df_social["Session_End"] - df_social["Session_Start"]).dt.total_seconds() / 60

# 2. High Exposure (if session > 45 mins)
df_social["High_Exposure"] = df_social["Session_Duration_Min"].apply(lambda x: 1 if x > 45 else 0)

# Preview updated DataFrame
df_social.head()

,User_ID,Platform,Session_Start,Session_End,Session_Duration_Min,High_Exposure
0,1102,TikTok,2023-05-22 08:44:00,2023-05-22 09:17:00,33.0,0
1,1435,Snapchat,2023-05-22 08:40:00,2023-05-22 09:58:00,78.0,1
2,1860,YouTube,2023-05-22 08:19:00,2023-05-22 08:43:00,24.0,0
3,1270,YouTube,2023-05-22 11:13:00,2023-05-22 11:54:00,41.0,0
4,1106,Instagram,2023-05-22 10:20:00,2023-05-22 10:47:00,27.0,0


In [254]:
df_social.to_excel("Transformed_Social_Media_Sessions.xlsx", index=False)